In [1]:
#!npm install react react-dom @xyflow/react @anywidget/react

In [2]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path(Path.cwd())))

In [3]:
!npx esbuild js/widget.jsx --minify --format=esm --bundle --outdir=pyironflow/static

⠙
  pyironflow/static/widget.js    1.7mb ⚠️
  pyironflow/static/widget.css  16.4kb

⚡ Done in 358ms
⠙

In [4]:
from pyiron_workflow import Workflow
#from typing import Optional
from pyironflow.pyironflow import PyironFlow
#from pyironflow.macro import get_node_instance_type

#from pyiron_workflow import Workflow
#from pyiron_workflow.channels import NotData
#from pyironflow.themes import get_color
#from pyironflow.macro import get_node_instance_type
#from pyironflow.wf_extensions import get_import_path, create_macro

import anywidget
import pathlib
import traitlets
import os
import json
import importlib
import typing


/home/jmittler/miniconda3/envs/pyironflow-forge/lib/python3.12/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [5]:
wf = Workflow('lammps_macro')

In [6]:
from pyiron_workflow import as_function_node, as_macro_node
import typing
from typing import Optional

@as_function_node()
def add_1(a: Optional[float | int] = None):
    return a+1


@as_function_node()
def part_1(a: float | int = 0):
    out = a+1
    return out

@as_function_node()
def part_2(a: float | int = 0):
    out = a+2
    return out

@as_function_node()
def part_3(a: float | int = 0):
    out = a+3
    return out

@as_function_node()
def part_4(a: float | int = 0, b: float | int = 0):
    out = a+b
    return out

@as_macro_node()
def macro_add(wf, a: int = 0):

    # imports

    wf.one = part_1(a)
    wf.two = part_2(wf.one)
    wf.three = part_3(wf.two)
    
    return wf.three

@as_macro_node()
def macro_addmore(wf, a: int = 0, b: int = 0):

    # imports

    wf.inp1 = part_1(a)
    wf.inp2 = part_2(b)
    wf.outp = part_4(wf.inp1, wf.inp2)
    
    return wf.outp


@as_macro_node()
def macro_visual(wf, a: int = 0, b: int = 0, c: int = 0):

    # imports

    wf.in1 = part_1(a)
    wf.calc1 = part_4(wf.in1, b)
    wf.calc2 = part_2(c)
    wf.out_node = part_4(wf.calc1, wf.calc2)
    
    return wf.out_node
    


wf.add1 = add_1()
wf.add2 = add_1(wf.add1)
wf.add3 = add_1()

wf.add5 = add_1()
wf.add6 = add_1()


#wf.macro2 = macro_add(wf.macro1)

wf.macro = macro_visual(wf.add2, wf.add3)
wf.macro2 = macro_addmore(wf.add5,wf.add6)

wf.add4 = add_1(wf.macro)




In [7]:
pf = PyironFlow([wf])
pf.gui

In [12]:
from pyironflow.wf_extensions import get_nodes


In [27]:
for k, v in wf.macro2.children.items():
    print(v)

inp1 (part_1):
Inputs ['a']
OutputsWithInjection ['out']
InputSignals ['run', 'accumulate_and_run']
OutputSignals ['ran', 'failed']
inp2 (part_2):
Inputs ['a']
OutputsWithInjection ['out']
InputSignals ['run', 'accumulate_and_run']
OutputSignals ['ran', 'failed']
outp (part_4):
Inputs ['a', 'b']
OutputsWithInjection ['out']
InputSignals ['run', 'accumulate_and_run']
OutputSignals ['ran', 'failed']
injected_GetAttr_3579870511310371271 (GetAttr):
Inputs ['obj', 'name']
OutputsWithInjection ['getattr']
InputSignals ['run', 'accumulate_and_run']
OutputSignals ['ran', 'failed']


In [34]:
for n in json.loads(pf.wf_widgets[0].gui.nodes):
    print(n["id"])

add1
add2
add3
add5
add6
macro
macro2
add4


In [41]:
node = "macro2"

for n in json.loads(pf.wf_widgets[0].gui.nodes):
    if n["id"] == node :
        print(n)

{'id': 'macro2', 'data': {'label': 'macro2', 'source_labels': ['outp'], 'target_labels': ['a', 'b'], 'import_path': '__main__.macro_addmore', 'target_values': [0, 0], 'target_types': ['int', 'int'], 'target_types_raw': ['int', 'int'], 'target_literal_values': [[], []], 'target_literal_types': [[], []], 'source_values': ['NotData'], 'source_types': ['None'], 'source_types_raw': ['Not Explicitly Defined'], 'failed': 'False', 'running': 'False', 'ready': 'False', 'cache_hit': 'False', 'python_object_id': 140567737876848}, 'position': {'x': 332, 'y': 52}, 'type': 'macroNode', 'style': {'padding': 5, 'background': 'rgba(234, 207, 159, 1)', 'borderRadius': '10px', 'width': '240PX', 'width_unitless': 240, 'height': '72px', 'height_unitless': 72}, 'targetPosition': 'left', 'sourcePosition': 'right', 'measured': {'width': 240, 'height': 72}}


In [69]:
node = "macro2"

for k, v in wf.children.items():
    if k == node:
        print(get_nodes(v, []))
        for k_sub, v_sub in wf.children.items():
    

[{'id': 'inp1', 'data': {'label': 'inp1', 'source_labels': ['out'], 'target_labels': ['a'], 'import_path': '__main__.part_1', 'target_values': [0], 'target_types': ['int-float'], 'target_types_raw': [['float', 'int']], 'target_literal_values': [None], 'target_literal_types': [None], 'source_values': ['NotData'], 'source_types': ['None'], 'source_types_raw': ['Not Explicitly Defined'], 'failed': 'False', 'running': 'False', 'ready': 'False', 'cache_hit': 'False', 'python_object_id': 140567737878432}, 'position': {'x': 0, 'y': 0}, 'type': 'customNode', 'style': {'padding': 5, 'background': '#a2ea9f', 'borderRadius': '10px', 'width': '240PX', 'width_unitless': 240, 'height': '56px', 'height_unitless': 56}, 'targetPosition': 'left', 'sourcePosition': 'right'}, {'id': 'inp2', 'data': {'label': 'inp2', 'source_labels': ['out'], 'target_labels': ['a'], 'import_path': '__main__.part_2', 'target_values': [0], 'target_types': ['int-float'], 'target_types_raw': [['float', 'int']], 'target_literal

In [47]:
wf.macro2

In [11]:
get_nodes(wf.macro2, [])

[{'id': 'inp1',
  'data': {'label': 'inp1',
   'source_labels': ['out'],
   'target_labels': ['a'],
   'import_path': '__main__.part_1',
   'target_values': [0],
   'target_types': ['int-float'],
   'target_types_raw': [['float', 'int']],
   'target_literal_values': [None],
   'target_literal_types': [None],
   'source_values': ['NotData'],
   'source_types': ['None'],
   'source_types_raw': ['Not Explicitly Defined'],
   'failed': 'False',
   'running': 'False',
   'ready': 'False',
   'cache_hit': 'False',
   'python_object_id': 140567737878432},
  'position': {'x': 0, 'y': 0},
  'type': 'customNode',
  'style': {'padding': 5,
   'background': '#a2ea9f',
   'borderRadius': '10px',
   'width': '240PX',
   'width_unitless': 240,
   'height': '56px',
   'height_unitless': 56},
  'targetPosition': 'left',
  'sourcePosition': 'right'},
 {'id': 'inp2',
  'data': {'label': 'inp2',
   'source_labels': ['out'],
   'target_labels': ['a'],
   'import_path': '__main__.part_2',
   'target_values'

In [62]:
i=1
node_list = get_nodes(wf.macro2, [])
for n in node_list:
    n["position"]["x"] = i
    n["position"]["y"] = i
    i +=1

#pf.wf_widgets[0].gui.nodes = json.dumps(node_list)

    

In [ ]:
wf.

In [58]:
node_list = get_nodes(wf.macro2, [])
node_list[0]["id"] = "bla"
node_list

[{'id': 'bla',
  'data': {'label': 'inp1',
   'source_labels': ['out'],
   'target_labels': ['a'],
   'import_path': '__main__.part_1',
   'target_values': [0],
   'target_types': ['int-float'],
   'target_types_raw': [['float', 'int']],
   'target_literal_values': [None],
   'target_literal_types': [None],
   'source_values': ['NotData'],
   'source_types': ['None'],
   'source_types_raw': ['Not Explicitly Defined'],
   'failed': 'False',
   'running': 'False',
   'ready': 'False',
   'cache_hit': 'False',
   'python_object_id': 140567737878432},
  'position': {'x': 0, 'y': 0},
  'type': 'customNode',
  'style': {'padding': 5,
   'background': '#a2ea9f',
   'borderRadius': '10px',
   'width': '240PX',
   'width_unitless': 240,
   'height': '56px',
   'height_unitless': 56},
  'targetPosition': 'left',
  'sourcePosition': 'right'},
 {'id': 'inp2',
  'data': {'label': 'inp2',
   'source_labels': ['out'],
   'target_labels': ['a'],
   'import_path': '__main__.part_2',
   'target_values':

In [16]:
json.loads(pf.wf_widgets[0].gui.nodes)

[{'id': 'add1',
  'data': {'label': 'add1',
   'source_labels': ['a+1'],
   'target_labels': ['a'],
   'import_path': '__main__.add_1',
   'target_values': [None],
   'target_types': ['int-float'],
   'target_types_raw': [['float', 'int', 'NoneType']],
   'target_literal_values': [None],
   'target_literal_types': [None],
   'source_values': ['NotData'],
   'source_types': ['None'],
   'source_types_raw': ['Not Explicitly Defined'],
   'failed': 'False',
   'running': 'False',
   'ready': 'False',
   'cache_hit': 'False',
   'python_object_id': 140567739399056},
  'position': {'x': 12, 'y': 232},
  'type': 'customNode',
  'style': {'padding': 5,
   'background': '#a2ea9f',
   'borderRadius': '10px',
   'width': '240PX',
   'width_unitless': 240,
   'height': '56px',
   'height_unitless': 56},
  'targetPosition': 'left',
  'sourcePosition': 'right',
  'measured': {'width': 240, 'height': 56}},
 {'id': 'add2',
  'data': {'label': 'add2',
   'source_labels': ['a+1'],
   'target_labels': [